In [ ]:
activities.to_json('user-activities.json', orient='records', lines=True) ##save as new line delimited json

In [ ]:
from google.cloud import bigquery

PROJECT='storyscore-356114' #replace with your project id
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service-key.json' ##replace it with path to your key file


def get_table_ref(client: bigquery.Client, dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    return table_ref


def create_load_job_config(job_type='WRITE_TRUNCATE'):
    return bigquery.LoadJobConfig(
    write_disposition=job_type,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True
    )

def upload_json(filename: str, dataset_id: str, table_id: str, config):
    print('uploading table: %s'%(table_id))
    client = bigquery.Client(project=PROJECT)
    table_ref = get_table_ref(client, dataset_id, table_id)
    with open(filename, "rb") as source_file:
        job = client.load_table_from_file(
            source_file, table_ref, job_config=config
        )  # Make an API request.
        job.result()  # Wait for the job to complete.;
    table = client.get_table(table_ref)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

def upload_user_activity(filename: str):
    dataset = 'github_user_activity'
    table_name = 'user_activity'
    config = create_load_job_config()
    upload_json(filename=filename, dataset_id=dataset, table_id=table_name, config= config)
    return


In [ ]:
upload_user_activity('./user-activities.json')